In [2]:
#!/usr/bin/env python3
import asyncio, aiohttp
from datetime import date
import pandas as pd

API_KEY   = "c5PobUQjaaMTHySILWqmWi9uyIDqYJBi"
BASE      = "https://financialmodelingprep.com/api/v3"
MAX_WORKERS, TIMEOUT_SEC, RETRIES = 5, 15, 3
N_BUSINESS_DAYS = 250

# ---------- Dates: strictly business days (Mon–Fri) ----------
def last_business_days(end: date, periods: int) -> pd.DatetimeIndex:
    # Mon–Fri only. If you want US market holidays too, replace with CustomBusinessDay(calendar=...)
    return pd.bdate_range(end=pd.Timestamp(end), periods=periods, freq="B")

# ---------- HTTP ----------
async def fetch_json(session, url, retries=RETRIES):
    for attempt in range(retries):
        try:
            async with session.get(url, timeout=TIMEOUT_SEC) as r:
                if r.status == 200:
                    return await r.json()
                if r.status in (429, 500, 502, 503, 504):
                    await asyncio.sleep(1.5 * (attempt + 1))
                else:
                    await asyncio.sleep(0.6)
        except Exception:
            await asyncio.sleep(0.6)
    raise RuntimeError(f"Failed GET {url}")

# ---------- Step 1: current members + change log ----------
async def load_spx_members_and_changes():
    async with aiohttp.ClientSession() as session:
        cur_url  = f"{BASE}/sp500_constituent?apikey={API_KEY}"
        hist_url = f"{BASE}/historical/sp500_constituent?apikey={API_KEY}"
        cur, hist = await asyncio.gather(
            fetch_json(session, cur_url),
            fetch_json(session, hist_url),
        )
    current = {r["symbol"] for r in cur if isinstance(r, dict) and r.get("symbol")}
    chg = pd.DataFrame(hist)
    if chg.empty:
        chg = pd.DataFrame(columns=["date", "symbol", "addedSecurity", "removedTicker"])
    else:
        keep = [c for c in ["date", "symbol", "addedSecurity", "removedTicker", "reason"] if c in chg.columns]
        chg = chg[keep].copy()
        chg["date"] = pd.to_datetime(chg["date"]).dt.date
    return current, chg

# ---------- Step 2: daily membership reconstruction (backward) ----------
def rebuild_daily_membership(current_set, changes_df, bdays: pd.DatetimeIndex):
    """
    Produces a dict: {date -> set(symbols)} for each business day in `bdays`.

    Logic:
      - Start at the most recent business day with today's *current_set*.
      - For each day going backwards:
          * Record today's membership.
          * Then UNDO any events that occurred on that date to get the previous day's set:
             - If a symbol was ADDED on D, it should NOT be in the set BEFORE D  -> remove it.
             - If a symbol was REMOVED on D, it SHOULD be in the set BEFORE D   -> add it.
    """
    bdays_desc = list(reversed(bdays))  # newest -> oldest
    min_day, max_day = bdays.min().date(), bdays.max().date()

    if not changes_df.empty:
        ev = changes_df[(changes_df["date"] >= min_day) & (changes_df["date"] <= max_day)]
    else:
        ev = pd.DataFrame(columns=["date", "symbol", "addedSecurity", "removedTicker"])
    events_by_date = {d: df for d, df in ev.groupby("date")}

    members_map = {}
    cur = set(current_set)

    for dt_ts in bdays_desc:
        d = dt_ts.date()
        # 1) Record membership for this date (after events of this date have taken effect)
        members_map[d] = set(cur)

        # 2) Prepare for the previous (earlier) date: undo today's events
        todays = events_by_date.get(d)
        if todays is not None:
            for _, r in todays.iterrows():
                sym = r.get("symbol")
                if not sym:
                    continue
                if str(r.get("addedSecurity", "")).strip():
                    # Added on d -> before d it wasn't a member
                    if sym in cur:
                        cur.remove(sym)
                if str(r.get("removedTicker", "")).strip():
                    # Removed on d -> before d it was still a member
                    cur.add(sym)

    return members_map  # keys = each business-day date

# ---------- Step 3: classifications (sector/subsector) ----------
async def fetch_classifications(symbols):
    sem = asyncio.Semaphore(MAX_WORKERS)

    async def one(session, sym):
        url = f"{BASE}/profile/{sym}?apikey={API_KEY}"
        for attempt in range(RETRIES):
            try:
                async with sem:
                    async with session.get(url, timeout=TIMEOUT_SEC) as r:
                        if r.status == 200:
                            js = await r.json()
                            row = (js[0] if isinstance(js, list) and js else {}) or {}
                            sector    = (row.get("sector") or "").strip() or "Unknown"
                            subsector = (row.get("subSector") or row.get("subsector") or row.get("industry") or "").strip() or "Unknown"
                            return sym, sector, subsector
                        if r.status in (429, 500, 502, 503, 504):
                            await asyncio.sleep(1.25 * (attempt + 1))
                        else:
                            await asyncio.sleep(0.6)
            except Exception:
                await asyncio.sleep(0.6)
        return sym, "Unknown", "Unknown"

    out = {}
    async with aiohttp.ClientSession() as session:
        tasks = [asyncio.create_task(one(session, s)) for s in symbols]
        for t in asyncio.as_completed(tasks):
            sym, sector, subsector = await t
            out[sym] = {"sector": sector, "subsector": subsector}
    return out

# ---------- Step 4: build daily sector/subsector portfolios ----------
def build_portfolios(bdays, members_map, classes):
    rows = []
    for dt_ts in bdays:  # oldest -> newest for stable ordering
        d = dt_ts.date()
        for sym in members_map.get(d, ()):
            cls = classes.get(sym, {})
            sector    = cls.get("sector", "Unknown") or "Unknown"
            subsector = cls.get("subsector", "Unknown") or "Unknown"
            rows.append(("sector::"    + sector,    sym, d))
            rows.append(("subsector::" + subsector, sym, d))
    df = pd.DataFrame(rows, columns=["portfolio", "ticker", "dt"])
    return df

# ---------- MAIN ----------
async def main():
    # A) Dates: strictly business days (no weekends)
    bdays = last_business_days(end=date.today(), periods=N_BUSINESS_DAYS)

    # B) Current members + change log (FMP)
    current_members, changes_df = await load_spx_members_and_changes()

    # C) Daily membership reconstruction (uses current+changes)
    daily_members = rebuild_daily_membership(current_members, changes_df, bdays)

    # D) Universe = all symbols that appeared on any of these days
    all_syms = sorted({s for sset in daily_members.values() for s in sset})

    # E) Fetch GICS sector/subsector once per symbol
    classes = await fetch_classifications(all_syms)

    # F) Build the daily portfolio rows FROM the reconstructed daily constituents
    daily_sector_subsector_portfolios = build_portfolios(bdays, daily_members, classes)

    print(daily_sector_subsector_portfolios.head(8))
    print(daily_sector_subsector_portfolios.tail(8))
    print(
        "rows:", len(daily_sector_subsector_portfolios),
        "| unique dates:", daily_sector_subsector_portfolios["dt"].nunique(),
        "| portfolios:", daily_sector_subsector_portfolios["portfolio"].nunique()
    )
    return daily_sector_subsector_portfolios

# ---------- RUNNER that works in both scripts and notebooks ----------
def run_async(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = None

    if loop and loop.is_running():
        # Notebook/REPL path
        try:
            import nest_asyncio
            nest_asyncio.apply()
        except ImportError as e:
            raise RuntimeError(
                "An event loop is already running (notebook). "
                "Either: (a) run `await main()` in a cell, or "
                "(b) `pip install nest_asyncio` and retry."
            ) from e
        return loop.run_until_complete(coro)
    else:
        # Regular script path
        return asyncio.run(coro)

# ===== Execute =====
if __name__ == "__main__":
    daily_sector_subsector_portfolios = run_async(main())
else:
    # If you're in a Jupyter cell, you can also do:
    # daily_sector_subsector_portfolios = await main()
    pass


                     portfolio ticker          dt
0      sector::Basic Materials    DOW  2024-10-24
1         subsector::Chemicals    DOW  2024-10-24
2          sector::Real Estate     WY  2024-10-24
3  subsector::REIT - Specialty     WY  2024-10-24
4   sector::Financial Services     RF  2024-10-24
5  subsector::Banks - Regional     RF  2024-10-24
6          sector::Real Estate    FRT  2024-10-24
7     subsector::REIT - Retail    FRT  2024-10-24
                                        portfolio ticker          dt
251500                        sector::Industrials    XYL  2025-10-08
251501          subsector::Industrial - Machinery    XYL  2025-10-08
251502                 sector::Financial Services    IVZ  2025-10-08
251503                subsector::Asset Management    IVZ  2025-10-08
251504             sector::Communication Services   META  2025-10-08
251505  subsector::Internet Content & Information   META  2025-10-08
251506                  sector::Consumer Cyclical     GM  2025-10-0

In [8]:
import duckdb
import pandas as pd

# --- Config ---
DB_PATH = "/Users/martingobbo/stock-dashboard/data/serving/analytics.duckdb"

# 1) Universe from the portfolios DF (FMP S&P constituents over 250 business days)
fmp_syms = (
    daily_sector_subsector_portfolios["ticker"]
    .dropna().astype(str).str.strip().str.upper().unique()
)
fmp_set = set(fmp_syms)

# 2) Read dim_ticker tickers
con = duckdb.connect(DB_PATH, read_only=True)
dim = con.execute("SELECT UPPER(TRIM(ticker)) AS ticker FROM dim_ticker").df()
con.close()
dim_set = set(dim["ticker"].dropna().astype(str))

# 3) Missing = in FMP set but not in dim_ticker
missing = sorted(fmp_set - dim_set)

print(
    f"FMP universe: {len(fmp_set)} | "
    f"dim_ticker: {len(dim_set)} | "
    f"missing: {len(missing)}"
)

missing_df = pd.DataFrame({"ticker": missing})
display(missing_df.head(50))  # preview first 50
# If you want the full list:
# display(missing_df)


FMP universe: 504 | dim_ticker: 514 | missing: 7


,ticker
0,AMTM
1,APP
2,BF-B
3,BRK-B
4,EME
5,HOOD
6,IBKR


In [4]:
# === Cell 2: Build sector/subsector portfolios and populate fact_portfolio_daily ===
import re
import pandas as pd
import duckdb

# ---- CONFIG: point to your DuckDB file ----
DB_PATH = "/Users/martingobbo/stock-dashboard/data/serving/analytics.duckdb"

# ---- 1) Normalize portfolio codes (sector_technology, subsector_chemicals_technology) ----
def slugify(s: str) -> str:
    s = (s or "").strip().lower()
    s = re.sub(r"[^0-9a-z]+", "_", s)
    return s.strip("_") or "unknown"

# daily_sector_subsector_portfolios has rows like:
#   portfolio: "sector::Technology" or "subsector::Chemicals"
#   ticker: "AAPL"
#   dt: date
df = daily_sector_subsector_portfolios.copy()

# Split kind/name
tmp = df.copy()
tmp["kind"] = tmp["portfolio"].str.split("::", n=1).str[0].str.lower()       # 'sector' or 'subsector'
tmp["name"] = tmp["portfolio"].str.split("::", n=1).str[1].fillna("Unknown")

# Pivot to get both sector and subsector for each (dt, ticker)
pivot = (
    tmp
    .pivot_table(index=["dt", "ticker"], columns="kind", values="name", aggfunc="first")
    .reset_index()
)

# Build assignment rows: two portfolios per (dt, ticker)
assign_rows = []
for r in pivot.itertuples(index=False):
    dt, ticker = r.dt, r.ticker
    sector_name    = getattr(r, "sector", "Unknown") or "Unknown"
    subsector_name = getattr(r, "subsector", "Unknown") or "Unknown"

    # sector code: sector_<sector>
    sector_code = f"sector_{slugify(sector_name)}"
    assign_rows.append((pd.to_datetime(dt).date(), ticker, sector_code))

    # subsector code: subsector_<subsector>_<sector>
    subsector_code = f"subsector_{slugify(subsector_name)}_{slugify(sector_name)}"
    assign_rows.append((pd.to_datetime(dt).date(), ticker, subsector_code))

assign = pd.DataFrame(assign_rows, columns=["dt", "ticker", "portfolio_code"]).drop_duplicates()

# ---- 2) Open DB, ensure tables exist ----
con = duckdb.connect(DB_PATH)

con.execute("""
CREATE TABLE IF NOT EXISTS dim_portfolio (
  portfolio_id   INTEGER PRIMARY KEY,
  portfolio_code TEXT UNIQUE NOT NULL
);
""")

# Create with the corrected column name; do NOT include market_cap.
con.execute("""
CREATE TABLE IF NOT EXISTS fact_portfolio_daily (
  portfolio_id  INTEGER NOT NULL,
  ticker_id     INTEGER NOT NULL,
  dt            DATE    NOT NULL,
  weight        DOUBLE,
  market_value  DOUBLE,   -- holds the ticker's market cap for that day
  PRIMARY KEY (portfolio_id, ticker_id, dt)
);
""")

# --- Schema migration (if an older column named market_cap exists, drop it) ---
cols_df = con.execute("PRAGMA table_info('fact_portfolio_daily')").df()
if "market_cap" in [c.lower() for c in cols_df["name"].tolist()]:
    con.execute("ALTER TABLE fact_portfolio_daily DROP COLUMN market_cap;")

# ---- 3) Ensure all portfolio_codes exist in dim_portfolio (assign new ids for new codes) ----
existing_df = con.execute("SELECT portfolio_code, portfolio_id FROM dim_portfolio").df()
existing_codes = set(existing_df["portfolio_code"]) if not existing_df.empty else set()
needed_codes = sorted(set(assign["portfolio_code"]) - existing_codes)

if needed_codes:
    max_id = 0 if existing_df.empty else int(existing_df["portfolio_id"].max())
    to_insert = [(max_id + i + 1, code) for i, code in enumerate(needed_codes)]
    con.executemany("INSERT INTO dim_portfolio (portfolio_id, portfolio_code) VALUES (?, ?)", to_insert)

# (Optional) sanity check
new_dim_count = con.execute("SELECT COUNT(*) FROM dim_portfolio").fetchone()[0]
print(f"dim_portfolio total rows: {new_dim_count}")

# ---- 4) Register helper DataFrames and build weights from market caps ----
# Map ticker -> ticker_id
ticker_map = con.execute("SELECT ticker, ticker_id FROM dim_ticker").df()

# Register temp views
con.register("assign", assign)
con.register("ticker_map", ticker_map)

# We’ll compute:
# - portfolio_id for each assignment (join dim_portfolio)
# - bring in fact_marketcap_daily for each (dt, ticker_id)
# - total per portfolio_id/day
# - weight = ticker_cap / total_cap
sql_weights = """
WITH a AS (
  SELECT 
    a.dt,
    lower(a.ticker) AS ticker_lc,
    a.portfolio_code
  FROM assign a
),
a2 AS (
  SELECT 
    a.dt,
    tm.ticker_id,
    dp.portfolio_id
  FROM a
  JOIN ticker_map tm
    ON lower(tm.ticker) = a.ticker_lc
  JOIN dim_portfolio dp
    ON dp.portfolio_code = a.portfolio_code
),
m AS (
  SELECT 
    a2.portfolio_id,
    a2.ticker_id,
    f.dt,
    f.market_cap
  FROM a2
  JOIN fact_marketcap_daily f
    ON f.ticker_id = a2.ticker_id
   AND f.dt = a2.dt
  WHERE f.market_cap IS NOT NULL
),
tot AS (
  SELECT portfolio_id, dt, SUM(market_cap) AS total_cap
  FROM m
  GROUP BY portfolio_id, dt
),
weights AS (
  SELECT
    m.portfolio_id,
    m.ticker_id,
    m.dt,
    m.market_cap,
    CASE WHEN tot.total_cap > 0 THEN m.market_cap / tot.total_cap ELSE NULL END AS weight
  FROM m
  JOIN tot
    ON tot.portfolio_id = m.portfolio_id AND tot.dt = m.dt
)
SELECT * FROM weights
"""

weights_df = con.execute(sql_weights).df()
print(
    f"Computed weights rows: {len(weights_df)} | "
    f"unique dates: {weights_df['dt'].nunique() if not weights_df.empty else 0} | "
    f"portfolios: {weights_df['portfolio_id'].nunique() if not weights_df.empty else 0}"
)

# ---- 5) Upsert into fact_portfolio_daily ----
# IMPORTANT: market_value := ticker's market_cap for that day
con.register("weights_df", weights_df)

con.execute("""
MERGE INTO fact_portfolio_daily AS dst
USING (
  SELECT 
    portfolio_id, ticker_id, dt, 
    weight,
    market_cap AS market_value
  FROM weights_df
) AS src
ON  dst.portfolio_id = src.portfolio_id
AND dst.ticker_id    = src.ticker_id
AND dst.dt           = src.dt
WHEN MATCHED THEN UPDATE SET
  weight       = src.weight,
  market_value = src.market_value
WHEN NOT MATCHED THEN INSERT
  (portfolio_id, ticker_id, dt, weight, market_value)
VALUES
  (src.portfolio_id, src.ticker_id, src.dt, src.weight, src.market_value);
""")

# Small summary
summary = con.execute("""
SELECT 
  COUNT(*) AS rows_written,
  COUNT(DISTINCT dt) AS days,
  COUNT(DISTINCT portfolio_id) AS portfolios
FROM fact_portfolio_daily
WHERE dt IN (SELECT DISTINCT dt FROM weights_df)
""").df()
print(summary)

# Clean up registered views
con.unregister("assign")
con.unregister("ticker_map")
con.unregister("weights_df")
con.close()


/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/pandas/core/strings/object_array.py:354: RuntimeWarning: coroutine 'main' was never awaited
  f = lambda x: new_pat.split(x, maxsplit=n)


dim_portfolio total rows: 129
Computed weights rows: 231950 | unique dates: 238 | portfolios: 128
   rows_written  days  portfolios
0        231950   238         128


In [7]:
# === Cell 3 (fixed): Inspect completeness across price, marketcap, and portfolio tables (last 250 days) ===
import duckdb
import pandas as pd

DB_PATH = "/Users/martingobbo/stock-dashboard/data/serving/analytics.duckdb"

con = duckdb.connect(DB_PATH, read_only=True)

sql = """
-- Universe: only Stocks, last 250 trading days based on price table's max date
WITH stocks AS (
  SELECT ticker_id, ticker
  FROM dim_ticker
  WHERE lower(ticker_type) = 'stock'
),
px_all AS (
  SELECT p.dt, p.ticker_id
  FROM fact_price_daily p
  JOIN stocks s USING (ticker_id)
),
maxd AS (
  SELECT max(dt) AS max_dt FROM px_all
),
px AS (
  SELECT *
  FROM px_all
  WHERE dt >= (SELECT max_dt FROM maxd) - INTERVAL 249 DAY
),

-- Collapse marketcap & portfolio to (dt, ticker_id, n_rows) so we can detect missing/dupes
mc AS (
  SELECT dt, ticker_id, COUNT(*) AS n_mc
  FROM fact_marketcap_daily
  GROUP BY 1,2
),
pf AS (
  SELECT dt, ticker_id, COUNT(*) AS n_pf
  FROM fact_portfolio_daily
  GROUP BY 1,2
),

-- Bring tickers for readability
px_labeled AS (
  SELECT px.dt, px.ticker_id, s.ticker
  FROM px
  JOIN stocks s USING (ticker_id)
),

-- Join price universe to marketcap & portfolio aggregates
joined AS (
  SELECT
    p.dt,
    p.ticker_id,
    p.ticker,
    COALESCE(mc.n_mc, 0) AS n_mc,
    COALESCE(pf.n_pf, 0) AS n_pf
  FROM px_labeled p
  LEFT JOIN mc ON mc.dt = p.dt AND mc.ticker_id = p.ticker_id
  LEFT JOIN pf ON pf.dt = p.dt AND pf.ticker_id = p.ticker_id
)

SELECT * FROM joined
ORDER BY dt, ticker;
"""

joined = con.execute(sql).df()

# --- Missing sets: present in price but missing in marketcap/portfolio (n == 0)
missing_mc = joined.loc[joined["n_mc"] == 0, ["dt", "ticker", "ticker_id"]].copy()
missing_pf = joined.loc[joined["n_pf"] == 0, ["dt", "ticker", "ticker_id"]].copy()

# --- Duplicates: combos where count != 1 (0 already captured as missing; here we show >1)
dupes_mc = joined.loc[joined["n_mc"] > 1, ["dt", "ticker", "ticker_id", "n_mc"]].copy()
dupes_pf = joined.loc[joined["n_pf"] > 1, ["dt", "ticker", "ticker_id", "n_pf"]].copy()

# --- Quick summaries
print("=== Coverage check (last 250 trading days; only ticker_type = 'Stock') ===")
print(f"Total (price universe) rows checked: {len(joined):,}")
print(f"Missing in fact_marketcap_daily: {len(missing_mc):,}")
print(f"Missing in fact_portfolio_daily: {len(missing_pf):,}")
print(f"Duplicates in fact_marketcap_daily (count > 1): {len(dupes_mc):,}")
print(f"Duplicates in fact_portfolio_daily (count > 1): {len(dupes_pf):,}")

# --- Show a few samples for each issue (keep prints manageable)
def preview(df, title, n=20):
    print(f"\n--- {title} (showing up to {n}) ---")
    if df.empty:
        print("None ✅")
    else:
        print(df.sort_values(["dt","ticker"]).head(n).to_string(index=False))

preview(missing_mc, "Missing in fact_marketcap_daily: (dt, ticker)")
preview(missing_pf, "Missing in fact_portfolio_daily: (dt, ticker)")
preview(dupes_mc,   "Duplicates in fact_marketcap_daily: (dt, ticker, n_mc)")
preview(dupes_pf,   "Duplicates in fact_portfolio_daily: (dt, ticker, n_pf)")

con.close()


=== Coverage check (last 250 trading days; only ticker_type = 'Stock') ===
Total (price universe) rows checked: 86,290
Missing in fact_marketcap_daily: 1,868
Missing in fact_portfolio_daily: 2,523
Duplicates in fact_marketcap_daily (count > 1): 0
Duplicates in fact_portfolio_daily (count > 1): 83,767

--- Missing in fact_marketcap_daily: (dt, ticker) (showing up to 20) ---
        dt ticker  ticker_id
2025-01-31   BF.B         57
2025-01-31  BRK.B         67
2025-01-31    RTX        401
2025-01-31   RVTY        402
2025-01-31   SBAC        403
2025-01-31   SBUX        404
2025-01-31   SCHW        405
2025-01-31    SHW        406
2025-01-31    SLB        408
2025-01-31   SMCI        409
2025-01-31   SNPS        411
2025-02-03   BF.B         57
2025-02-03  BRK.B         67
2025-02-03    RTX        401
2025-02-03   RVTY        402
2025-02-03   SBAC        403
2025-02-03   SBUX        404
2025-02-03   SCHW        405
2025-02-03    SHW        406
2025-02-03    SLB        408

--- Missing in